In [1]:
%run Accuracy_Module.py
%run DataLoading.py
%run load_and_organize_dataset.py
%run training.py

In [2]:
squeezenet = models.squeezenet1_0()

In [3]:
train_data, valid_data, test_data = load_dataset(64)

In [4]:
squeezenet.cuda()
squeezenet.eval()

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): Fire(
      (squeeze): Conv2d(128, 32, kerne

In [9]:
#SqueezeNet model
class SqueezeNet_regression(nn.Module):
    def __init__(self):
        super(SqueezeNet_regression, self).__init__()
        self.name = "SqueezeNet_regression"
        self.fc1 = nn.Linear(73728, 16384)
        self.fc2 = nn.Linear(16384, 4096)
        self.fc3 = nn.Linear(4096, 1024)
        self.fc4 = nn.Linear(1024, 1)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = x.cuda()
        x = squeezenet.features(x)
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        x = x.squeeze(1)
        
        return x

In [10]:
squeezeNet_regression = SqueezeNet_regression()
squeezeNet_regression.cuda()

RuntimeError: CUDA out of memory. Tried to allocate 4.50 GiB (GPU 0; 15.90 GiB total capacity; 1.65 GiB already allocated; 512.81 MiB free; 31.80 MiB cached)

In [ ]:
train_regression(train_data, valid_data, squeezeNet_regression, model_name = 'SqueezeNet', batch_size = 64, num_epochs = 20)